In [ ]:
#@title core
import os
import pickle
import torch
import torchvision.transforms as transforms
import sys
repo_path = '/Users/szymonlukiewicz/StudioProjects/psych_gen_app/content'

def setup_stylegan():
    """
    Sets up the StyleGAN environment by installing required packages, cloning the necessary GitHub repository,
    mounting Google Drive (if in Colab), downloading additional files, and loading the StyleGAN model.

    Returns:
        G (torch.nn.Module): The StyleGAN generator model.
        face_w (torch.Tensor): A tensor of sample latent vectors.
    """
    global repo_path
    # Change directory to content
    os.chdir(repo_path+'')

    # Download additional files
    os.system('gdown 1O79M5F5G3ktmt1-zeccbJf1Bhe8K9ROz')
    if not os.path.exists(repo_path+'/omi'):
        os.system('git clone https://github.com/jcpeterson/omi')
        os.system('unzip content/omi/attribute_ratings.zip')

    # Add necessary paths to sys.path
    sys.path.append(repo_path+'/psychGAN/stylegan3')
    sys.path.append(repo_path+'/psychGAN')
    os.chdir(repo_path+'/psychGAN')

    # Download the StyleGAN model file if not present
    model_path = repo_path+"/psychGAN/stylegan2-ffhq-1024x1024.pkl"
    if not os.path.exists(model_path):
    #   !rm content/psychGAN/stylegan2-ffhq-1024x1024.pkl
      !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-1024x1024.pkl

    # Load the StyleGAN model
    device = torch.device('mps')
    with open(model_path, 'rb') as fp:
        G = pickle.load(fp)['G_ema'].to(device)

    # Compute the average latent vector
    all_z = torch.randn([1, G.mapping.z_dim], device=device)
    face_w = G.mapping(all_z, None, truncation_psi=0.5)

    return G, face_w, device
G, face_w, device = setup_stylegan()



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Downloading...
From: https://drive.google.com/uc?id=1O79M5F5G3ktmt1-zeccbJf1Bhe8K9ROz
To: /Users/szymonlukiewicz/StudioProjects/psych_gen_app/content/coords_wlosses.h5
100%|██████████| 3.45M/3.45M [00:00<00:00, 3.72MB/s]


--2025-07-17 15:46:59--  https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-1024x1024.pkl
Translacja api.ngc.nvidia.com (api.ngc.nvidia.com)... 44.236.189.21, 52.40.32.32
Łączenie się z api.ngc.nvidia.com (api.ngc.nvidia.com)|44.236.189.21|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 302 Found
Lokalizacja: https://xfiles.ngc.nvidia.com/org/nvidia/team/research/models/stylegan2/versions/1/files/stylegan2-ffhq-1024x1024.pkl?ssec-algo=AES256&versionId=eumtJDh9E9G.kBYQtdwU2yJWZ.7WcykK&ssec-key=oJ8w89oBCKl72NGPN0SHT8xB%2BhENKLwT5gSXBqL4Q85yN%2BlQh6jssaWel6WJmm4Bz21Tz830yW2NMyVkZ1Tf%2FpDh6KeDjoHnd1WwEIpJcNOWd%2BBCjlNjc1SLdGSi9vOPcKFMnAimeksdw8vJM7QdMW7XdChgc9FEyLYEdhSdZicxfkwWlIRgQic7GTQbINDogZLBVs5GEdDvzJM6Q4tm9xIIM5tZJRV9ujrW3gFRsQXYoWJcT9Xg1BeIlSnOqP7CtNNvbnJI9RfbL4CN14sPyowr6hcA8TSFXic4mm8tS8MyvfsBvq0yh8lwj3cK2s09nnfzIf%2FMyTQJqkg7y6z1huUDYfDnBMRl5koSJy2iLAk%2Fc1aXnCyBrqD9y0QbmhaODmUhKcYIxng9wyGRt7yZX0lR3zkddflkaghVOItPy

In [29]:
#@title utils
import numpy as np
import PIL.Image
import IPython.display
from PIL import Image, ImageDraw
from math import ceil
from io import BytesIO
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import torch
import torchvision.transforms.functional as TF

def listify(x):
    """
    Converts a single element or a pandas DataFrame/Series to a list.
    If the input is already a list, it returns the input unmodified.

    Args:
        x: The input to be listified.

    Returns:
        list: A list of the input elements.
    """
    if isinstance(x, (list, pd.DataFrame, pd.Series)):
        return list(x)
    return [x]

def display_image(image_array, format='png', jpeg_fallback=True):
    """
    Displays an image in IPython.

    Args:
        image_array: A numpy array representing the image.
        format: The format of the image to display.
        jpeg_fallback: Whether to fall back to JPEG if the image is too large.

    Returns:
        The IPython.display object.
    """
    image_array = np.asarray(image_array, dtype=np.uint8)
    str_file = BytesIO()
    PIL.Image.fromarray(image_array).save(str_file, format)
    im_data = str_file.getvalue()
    try:
        return IPython.display.display(IPython.display.Image(im_data))
    except IOError as e:
        if jpeg_fallback and format != 'jpeg':
            print(f'Warning: image was too large to display in format "{format}"; trying jpeg instead.')
            return display_image(image_array, format='jpeg')
        else:
            raise

def create_image_grid(images, scale=1, rows=1):
    """
    Creates a grid of images.

    Args:
        images: A list of PIL.Image objects.
        scale: The scale factor for each image.
        rows: The number of rows in the grid.

    Returns:
        A single PIL.Image object containing the grid of images.
    """
    w, h = images[0].size
    w, h = int(w * scale), int(h * scale)
    height = rows * h
    cols = ceil(len(images) / rows)
    width = cols * w
    canvas = PIL.Image.new('RGBA', (width, height), 'white')
    for i, img in enumerate(images):
        img = img.resize((w, h), PIL.Image.ANTIALIAS)
        canvas.paste(img, (w * (i % cols), h * (i // cols)))
    return canvas

def dot_product(x, y):
    """
    Computes the normalized dot product of two vectors.

    Args:
        x, y: The vectors to compute the dot product of. Can be file paths or numpy arrays.

    Returns:
        The normalized dot product of x and y.
    """
    x = np.load(x) if isinstance(x, str) else x
    y = np.load(y) if isinstance(y, str) else y
    x_norm = x[1] if len(x.shape) > 1 else x
    y_norm = y[1] if len(y.shape) > 1 else y
    return np.dot(x_norm / np.linalg.norm(x_norm), y_norm / np.linalg.norm(y_norm))

def read(target, passthrough=True):
    """
    Transforms a path or array of coordinates into a standard format.

    Args:
        target: A path to the coordinate file or a numpy array.
        passthrough: If True, returns the target if it cannot be transformed.

    Returns:
        Transformed target or original target based on passthrough.
    """
    if target is None:
        return 0
    if isinstance(target, PIL.Image.Image):
        return None
    if isinstance(target, str):
        try:
            target = np.load(target)
        except:
            return target if passthrough else None
    if list(target.shape) == [1, 18, 512] or target.shape[0] == 18 or passthrough:
        return target
    if target.shape[0] in [1, 512]:
        return np.tile(target, (18, 1)) if isinstance(target, np.ndarray) else torch.tile(target, (18, 1))
    return target

def show_faces(target, add=None, subtract=False, plot=True, grid=True, rows=1, labels = None, device='cuda:0'):
    """
    Displays or returns images of faces generated from latent vectors.

    Args:
        target: Latent vectors or paths to images. Can be a string, np.array, or list thereof.
        add: Latent vector to add to the target. Can be None, np.array, or list thereof.
        subtract: If True, subtracts 'add' from 'target'.
        plot: If True, plots the images using matplotlib.
        grid: If True, displays images in a grid.
        rows: Number of rows in the grid.
        device: Device for PyTorch operations.
        G: The StyleGAN generator model.

    Returns:
        PIL images or None, depending on the 'plot' argument.
    """
    transform = Compose([
        Resize(512),
        lambda x: torch.clamp((x + 1) / 2, min=0, max=1)
    ])

    target, add = listify(target), listify(add)
    to_generate = [read(t, False) for t in target if read(t, False) is not None]

    if add[0] is not None:
        if len(add) == len(target):
            to_generate_add = [t + read(a) for t, a in zip(target, add)]
            to_generate_sub = [t - read(a) for t, a in zip(target, add)]
        else:
            to_generate_add = [t + read(add[0]) for t in target]
            to_generate_sub = [t - read(add[0]) for t in target]
        to_generate = [m for pair in zip(to_generate_sub, to_generate, to_generate_add) for m in pair] if subtract else [m for pair in zip(to_generate, to_generate_add) for m in pair]

    other = [PIL.Image.open(t) for t in target if isinstance(t, str) and not '.npy' in t]
    other += [t for t in target if isinstance(t, PIL.Image.Image)]
    for im in target:
        try:
            other += [TF.to_pil_image(transform(im))]
        except:
            pass

    images_pil = []
    if len(to_generate) > 0:
        global G
        with torch.no_grad():
            face_w = torch.tensor(to_generate, device=device)
            images = G.synthesis(face_w.view(-1, 18, 512))
            images_pil = [TF.to_pil_image(transform(im)) for im in images]

    images_pil += [(t) for t in other]

    if plot:
        display_images(images_pil, grid, rows, labels=labels)
    else:
        return create_image_grid(images_pil, rows=rows) if grid else images_pil

from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFont
import urllib.request
import functools
import io

import requests
from io import BytesIO

from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
import os
import cv2


def add_label_to_image(image, label, position=(10, 10), font_size=20):
    """
    Adds a label with a black stroke to an image at the specified position.

    Args:
        image: PIL.Image object.
        label: Text to add to the image.
        position: Tuple specifying the position to place the text.
        font_size: Size of the font.

    Returns:
        PIL.Image object with text added.
    """
    draw = ImageDraw.Draw(image)

    # You can use a system font or a bundled .ttf file
    font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
    font = ImageFont.truetype(font_path, font_size)

    # Get the bounding box for the text
    bbox = draw.textbbox(position, label, font=font)
    text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]

    # Adjust position based on the text height
    position = (position[0], position[1] - text_height*.5)

    # Outline (stroke) parameters
    stroke_width = 2
    stroke_fill = "black"

    # Draw text with outline
    draw.text(position, label, font=font, fill="white", stroke_width=stroke_width, stroke_fill=stroke_fill, textlength = text_width)

    return image



def display_images(images, grid, rows, labels):
    """
    Helper function to display images using matplotlib, with optional labels on each image.

    Args:
        images: A list of PIL.Image objects.
        grid: If True, displays images in a grid.
        rows: Number of rows in the grid.
        labels: List of labels for each image; if provided, labels will be added to images.
    """
    if labels:
        images = [add_label_to_image(im.copy(), lbl) for im, lbl in zip(images, labels)]

    if grid and len(images) > 1:
        cols = (len(images) + rows - 1) // rows  # Compute number of columns needed
        fig, axs = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
        axs = axs.flatten()  # Flatten the array of axes for easier iteration
        for idx, (im, ax) in enumerate(zip(images, axs)):
            ax.imshow(im)
            ax.axis('off')  # Hide axes
        plt.tight_layout()
        plt.show()
    else:
        for idx, im in enumerate(images):
            plt.figure(figsize=(5, 5))
            plt.imshow(im)
            plt.axis('off')
            plt.show()



In [44]:
#@title init
%cd content/psychGAN
import pickle
import torch
import torch.nn.functional as F
import sys
import shutil
import torchvision.transforms.functional as TF
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from sklearn.model_selection import train_test_split
import csv
import numpy as np
import pandas as pd
import PIL.Image
from PIL import Image, ImageDraw
import IPython.display
import imageio
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
sys.path.append(repo_path+'/psychGAN/stylegan3')
sys.path.append(repo_path+'/psychGAN')

import sys

import io
import os, time
import pickle
import shutil
import numpy as np
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import copy
import imageio
import unicodedata
import re
from PIL import Image
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
# from google.colab import files
from time import perf_counter

from stylegan3.dnnlib.util import open_url
df = pd.read_hdf(repo_path+'/coords_wlosses.h5')

[Errno 2] No such file or directory: 'content/psychGAN'
/Users/szymonlukiewicz/StudioProjects/psych_gen_app


In [48]:
#@title model
import torch
import numpy as np
from torchvision.transforms import Compose, Resize
import torchvision.transforms.functional as TF
from typing import List

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Model Definition
import torch
from torch import nn
class MultiTargetRegressor(nn.Module):
    def __init__(self, latent_dim, target_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(latent_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            # nn.Dropout(0.1),  # Removed feature dropout

            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, target_dim),
            # Consider an output activation here if appropriate for your target data
        )

    def forward(self, x):
        return self.network(x)

class EnsembleRegressor(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        return torch.mean(torch.stack(outputs), dim=0)
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from torchvision.transforms import Compose, Resize
import torchvision.transforms.functional as TF
from typing import List, Dict, Union, Tuple
from torchdiffeq import odeint

# Assume previous class definitions for VectorFieldTransformer, RatingODE, etc., are available.

class StyleGANFlowBackend:
    def __init__(self,
                 generator: nn.Module,
                 flow_models: Dict[str, nn.Module],
                 regression_models: Dict[str, nn.Module],
                 trait_stats_df: pd.DataFrame):
        """
        Initializes the StyleGANFlowBackend for flow-based manipulations.
        """
        self.device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
        print(f"✅ StyleGANFlowBackend initialized on device: {self.device}")

        self.G = generator.to(self.device).eval()
        self.flow_models = {k: m.to(self.device).eval() for k, m in flow_models.items()}
        self.models = {k: m.to(self.device).eval() for k, m in regression_models.items()}
        self.dimension_names = [*self.flow_models.keys()]
        # Handling for trait_stats_df if it's provided
        if trait_stats_df is not None and not trait_stats_df.empty:
            self.df = trait_stats_df
            self.m, self.std = self.df[self.dimension_names].mean(), self.df[self.dimension_names].std()
        else:
            self.df = pd.DataFrame()


    def to_w(self, target: Union[str, Path, np.ndarray, torch.Tensor]) -> torch.Tensor:
        """Converts various input types to a correctly shaped w latent tensor."""
        if isinstance(target, (str, Path)):
            target = np.load(target) if ".npy" in str(target) else torch.load(target)
        if isinstance(target, np.ndarray):
            target = torch.tensor(target, dtype=torch.float32, device=self.device)
        if target.dim() < 2: target = target.reshape(1, 1, 512)
        if target.dim() == 2: target = target.unsqueeze(0)
        if target.shape[1] == 1: target = target.repeat(1, 18, 1)
        return target.to(self.device).reshape(-1, 18, 512)

    def _to_pil(self, images_tensor: torch.Tensor):
        """Post-processes and converts a tensor of images to a list of PIL Images."""
        transform = Compose([Resize(512), lambda x: torch.clamp((x + 1) / 2, 0, 1)])
        return [TF.to_pil_image(img) for img in transform(images_tensor.cpu())]

    def _predict_ratings(self, w_tensor: torch.Tensor) -> Dict[str, np.ndarray]:
        """Predicts ratings for all registered models from a w tensor."""
        w0 = w_tensor[:, 0, :]
        return {label: model(w0).cpu().numpy() for label, model in self.models.items()}

    def _calculate_trajectory(self,
                              target_dim: str,
                              max_strength: float,
                              n_levels: int,
                              initial_w_for_flow: torch.Tensor,
                              max_steps: int) -> torch.Tensor:
        """
        Calculates a single trajectory and returns the latent codes at each level.
        """
        if n_levels == 0:
            return initial_w_for_flow.unsqueeze(0)

        is_forward = max_strength >= 0
        flow_field = self.flow_models[target_dim]

        # Define the time points for the ODE solver to evaluate
        t_span = torch.linspace(0, abs(max_strength), n_levels + 1, device=self.device)

        # Define the ODE function for the solver
        ode_func = lambda t, w: (1 if is_forward else -1) * flow_field(w)

        print(f"🚀 Computing {'forward' if is_forward else 'backward'} trajectory for '{target_dim}'...")

        # Use a simplified solver call
        trajectory_w0 = odeint(ode_func, initial_w_for_flow, t_span, method='rk4')

        return trajectory_w0

    def _generate_image_and_meta(self, w0: torch.Tensor, base_w: torch.Tensor, initial_ratings: dict, config: dict) -> Tuple:
        """Helper to generate an image and its metadata from a w0 vector."""
        manipulated_w = base_w.clone()
        manipulated_w[:, :, :] = w0.unsqueeze(1) # Apply to all layers

        if config.get('preserve_identity', True):
            style_layers = range(config.get('latents_from', 0), config.get('latents_to', 18), 2)
            manipulated_w[:, style_layers, :] = base_w[:, style_layers, :]

        image = self._to_pil(self.G.synthesis(manipulated_w))[0]
        final_ratings = self._predict_ratings(manipulated_w)
        distance = torch.norm(manipulated_w[:, 0, :] - base_w[:, 0, :], dim=-1).cpu().numpy()

        metadata = {
            "initial_ratings": initial_ratings,
            "final_ratings": final_ratings,
            "distance": distance
        }
        return image, metadata


    @torch.no_grad()
    def __call__(self, config: Dict, w: torch.Tensor = None) -> Tuple[List[List], List[List]]:
        # --- 1. Unpack Config and Initialize ---
        dims_config = config["manipulated_dimensions"]
        num_faces = config.get("num_faces", 1)
        truncation = config.get("truncation_psi", 0.7)
        max_steps = config.get("max_steps", 100)

        base_faces_w = self.to_w(w) if w is not None else self.G.mapping(torch.randn([num_faces, self.G.z_dim], device=self.device), None, truncation_psi=truncation)

        all_image_grids, all_metadata_grids = [], []

        # --- 2. Process each base face ---
        for i in range(num_faces):
            base_w = base_faces_w[i:i+1]
            w_for_flow = base_w[:, 0, :]
            initial_ratings = self._predict_ratings(base_w)

            # --- 3. Calculate Trajectories ---
            dim1_cfg = dims_config[0]
            traj1_w0 = self._calculate_trajectory(dim1_cfg['name'], dim1_cfg['strength'], dim1_cfg['n_levels'], w_for_flow, max_steps)

            n_levels1 = dim1_cfg['n_levels']

            # --- 4. Handle 1D or 2D manipulation ---
            if len(dims_config) == 1:
                # --- 4a. Single Dimension (One Row) ---
                n_levels2 = 0
                image_grid = [[None for _ in range(n_levels1 + 1)]]
                metadata_grid = [[None for _ in range(n_levels1 + 1)]]

                for c in range(n_levels1 + 1):
                    w0 = traj1_w0[c]
                    img, meta = self._generate_image_and_meta(w0, base_w, initial_ratings, config)
                    image_grid[0][c] = img
                    meta["manipulation"] = f"{dim1_cfg['name']}_{c}"
                    metadata_grid[0][c] = meta

            else:
                # --- 4b. Two Dimensions (Grid) ---
                dim2_cfg = dims_config[1]
                traj2_w0 = self._calculate_trajectory(dim2_cfg['name'], dim2_cfg['strength'], dim2_cfg['n_levels'], w_for_flow, max_steps)
                n_levels2 = dim2_cfg['n_levels']

                image_grid = [[None for _ in range(n_levels2 + 1)] for _ in range(n_levels1 + 1)]
                metadata_grid = [[None for _ in range(n_levels2 + 1)] for _ in range(n_levels1 + 1)]

                # Fill the first column (Dimension 1)
                for r in range(n_levels1 + 1):
                    w0 = traj1_w0[r]
                    img, meta = self._generate_image_and_meta(w0, base_w, initial_ratings, config)
                    image_grid[r][0] = img
                    meta["manipulation"] = f"{dim1_cfg['name']}_{r}"
                    metadata_grid[r][0] = meta

                # Fill the rest of the first row (Dimension 2), starting from the second element
                for c in range(1, n_levels2 + 1):
                    w0 = traj2_w0[c]
                    img, meta = self._generate_image_and_meta(w0, base_w, initial_ratings, config)
                    image_grid[0][c] = img
                    meta["manipulation"] = f"{dim2_cfg['name']}_{c}"
                    metadata_grid[0][c] = meta

            all_image_grids.append(image_grid)
            all_metadata_grids.append(metadata_grid)

        return all_image_grids, all_metadata_grids

if not os.path.exists("final_models.zip"):
  !gdown 1pPjOd-mx-d-vOw1QR_lpJoJmLAGdkI3W
  !unzip final_models.zip
if not "models" in dir():
  all_labels = [col for col in df.columns if col not in ['Unnamed: 0', 'stimulus', 'loss', 'dlatents']]

  models = [EnsembleRegressor([MultiTargetRegressor(512,1) for _ in range(8)]) for label in all_labels]
  for m,l in zip(models,all_labels):
    m.load_state_dict(torch.load(f"final_models/ensemble_{l}.pt", map_location='mps'))

In [49]:
import torch
import torch.nn as nn

class ControlGradientModel(nn.Module):
    """
    A wrapper class that computes the gradient of a control model's output
    with respect to its input latent vector 'w'.

    This allows a standard regression model (e.g., for age or gender) to act
    as a "flow" model within the StyleGANFlowBackend, where the "flow" is
    the direction of the gradient.
    """
    def __init__(self, control_model: nn.Module):
        """
        Initializes the ControlGradientModel.

        Args:
            control_model (nn.Module): A pre-trained model that takes a latent
                                       vector `w` and outputs a scalar value
                                       (e.g., predicted age).
        """
        super().__init__()
        self.model = control_model
        # Ensure the model is in evaluation mode as we only need it for inference.
        self.model.eval()

    def forward(self, w: torch.Tensor, **kwargs) -> torch.Tensor:
        """
        Computes and returns the gradient of the control model's output.

        This method is called by the NeuralODE solver, which expects a direction
        vector (the gradient) for each point 'w' in the latent space.

        Args:
            w (torch.Tensor): A batch of input latent vectors of shape [N, 512].
            **kwargs: Accepts and ignores additional arguments (like 'ratings') that
                      might be passed by the ODE wrapper for compatibility.

        Returns:
            torch.Tensor: The computed gradient for each latent vector in the batch,
                          representing the direction of steepest ascent for the attribute.
        """
        # The gradient calculation needs to be within a torch.enable_grad() context.
        with torch.enable_grad():
            # Detach 'w' from any previous computation graph and enable gradient tracking.
            w_for_grad = w.detach().clone().requires_grad_(True)

            # Perform a forward pass through the control model (e.g., get age prediction).
            output = self.model(w_for_grad)

            # Sum the output to create a scalar value. Calling .backward() on this
            # scalar computes the gradient of the sum with respect to w_for_grad.
            # This is equivalent to computing the gradient for each item in the batch independently.
            torch.sum(output).backward()

            # The gradient is now stored in .grad attribute of the input tensor.
            # We detach it from the graph before returning.
            gradient = w_for_grad.grad.detach()

        return gradient




# 1. Create a dictionary of the new gradient-based "flow" models.
control_flow_models = {
    name: ControlGradientModel(model)
    for name, model in zip(all_labels, models)
}

# 2. Combine the original learned flow models with the new control flow models.
#    This creates a unified dictionary of all available manipulations.
all_flow_models = {
    **control_flow_models # Add age, gender, happy
}

# 3. Combine all regression models for metadata prediction.
all_regression_models = {
    **{name: model for name, model in zip(all_labels, models)}
}

# 4. Instantiate the backend with all models.
backend = StyleGANFlowBackend(
    generator=G,
    flow_models=all_flow_models,
    regression_models=all_regression_models,
    trait_stats_df=df
)

✅ StyleGANFlowBackend initialized on device: mps


# Frontend

In [50]:
%cd /Users/szymonlukiewicz/StudioProjects/psych_gen_app/build

/Users/szymonlukiewicz/StudioProjects/psych_gen_app/build


In [ ]:
from flask import Flask, send_from_directory, jsonify, request
import os
import io
import base64
import time
import json

app = Flask(__name__, static_folder='web')


@app.route('/')
def index():
    return send_from_directory(app.static_folder, 'index.html')

@app.route('/<path:path>')
def static_files(path):
    return send_from_directory(app.static_folder, path)

def parse_config(conf):
    """
    Zamienia przychodzący JSON na słownik akceptowany przez StyleGANBackend.
    • Dla każdego wymiaru zwraca *listę py-floatów*, a nie tablicę NumPy.
      Dzięki temu dalsze mnożenie (float * Tensor) działa bez błędu.
    """
    if isinstance(conf, str):
        conf = json.loads(conf)
    print(json.dumps(conf, indent=2))

    latents_from, latents_to = {"both": (0,18), "color": (9,18), "shape":(0,9)}[conf.pop("mode", "both")]

    dim1 = conf["manipulated_dimensions"][0]
    conf["manipulated_dimensions"] = [dim1["name"]]
    # Note: np.linspace includes start and end points
    conf["strengths"] = [*np.linspace(-1*dim1["strength"], dim1["strength"], dim1["n_levels"])]
    print(conf["strengths"])
    conf["steps"] = min(5,conf.pop("max_steps", 5))
    conf["latents_from"] = latents_from
    conf["latents_to"] = latents_to
    # dims = conf["manipulated_dimensions"]
    # conf["manipulated_dimensions"] = [d["name"] for d in dims]

    # # ← kluczowa linia: tolist() zamienia ndarray na zwykłą listę floatów
    # if (len(dims) == 1):
    #   conf["strengths"] = [
    #       *np.linspace(-1*dims[0]["strength"], dims[0]["strength"], dims[0]["n_levels"])
    #   ]
    # elif (len(dims) == 2):
    #      conf["strengths"] = [
    #       *np.linspace(-1*dims[0]["strength"], dims[0]["strength"], dims[0]["n_levels"])
    #       *np.linspace(-1*dims[1]["strength"], dims[1]["strength"], dims[1]["n_levels"])
    #   ]
    print(conf)
    return conf

@app.route('/images', methods=['POST'])
def convert_images():
      config = request.json
      # config = parse_config(config)
      config["num_faces"]=1
      config["return_metadata"]=True

      images_pil, metadata = backend(config)

      image_array = images_pil[0]
      converted_images = []
      for i in range(len(image_array)):
        for j in range(len(image_array[i])):
          img = image_array[i][j]
          if img is None:
            continue
          # Convert PIL Image to bytes
          img_byte_arr = io.BytesIO()
          img.save(img_byte_arr, format='PNG')
          img_byte_arr = img_byte_arr.getvalue()

          # Encode bytes to base64 string
          image_array[i][j]=(base64.b64encode(img_byte_arr).decode('utf-8'))

      print(image_array)
      return image_array



if __name__ == '__main__':
    app.run()#debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2025 16:07:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /flutter.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /manifest.json HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /favicon.png HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /icons/Icon-192.png HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /flutter_service_worker.js?v=1247906683 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /main.dart.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /flutter_bootstrap.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /assets/AssetManifest.bin.json HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:09] "GET /assets/FontManifest.json HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 16:07:10] "GET /assets/fonts/MaterialIcons-Regular.otf HTTP/1.1" 200 -


🚀 Computing forward trajectory for 'dominant'...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


🚀 Computing forward trajectory for 'dominant'...🚀 Computing forward trajectory for 'dominant'...

🚀 Computing forward trajectory for 'dominant'...


...

[['...']]
🚀 Computing forward trajectory for 'trustworthy'...


127.0.0.1 - - [17/Jul/2025 16:12:42] "POST /images HTTP/1.1" 200 -


[['...']]


127.0.0.1 - - [17/Jul/2025 16:17:16] "POST /images HTTP/1.1" 200 -


[['...', None, None, None, None, None], ['iVBORw0KGgoAAAANSUhEUgAAAgAAAAIACAIAAAB7GkOtAAAf4UlEQVR4nO3debQlVX3o8W9BNygypRGZHiDBxRRmAziANIgkBo0y+KKCQwSCSYgi8uRhiEHwiUEJRDHIbDTKlIAokgBBBAwiQcKwGEVtAmITBkEkrXRDvT9On8u590w17Ko6p/b3s+5afU6dqr1/fRt+v7P3riFJ0xRJUsvtDtf3vE2BFZqKRZJUvR/DVpDMzv7LhS8AS+AMeCckFfysBF8NHrGqd0U1/z1MyM91Tf96peE2hd8DLur5L3ZGkmsKKBm/y4T6HHy06Rha7wD456ZjmHAH+Q1GtUsGZe6UFyAZPwI4Y1DdmAp/Cmn3x+xfg3/q+YUX/rm76b9FMQ9l+9uZ/UM4rt4B3jVN/32Le3R53k5h574PE2De6OOnLun3+g94DlZqOgzlsmVncSqPe+Bo+Na43faEHWELWBvOg2/M/nQnuArWzNm16nIjvL6hrvca/tFm8PfwxvpiGWlkul558CEjpoDugO1KRjQBvgN7NB2DpNCOg0821PWJ8H8b6nqocV/W536ejj0LaFtI4SvlwqrZYbBk9qDb7C+10XHDZ9oOCtH+bnAWPDWo/anL/sDLBx+YfRH4cVg7e0QV2xBOhAObDkOSGpdtrn7WXp0RQL6zgOb4Kry38MGZbQe3Vd+LJE2vDDVgQAEodR3AeyCFWwsduwb8AXwbboS74Suw6uwdTukOt24rE6IkCeDY/k2lRgCzGsq851lwyJCPjoTr4GWwKZwXJCxJikDeWaAAI4Beb8u22xbwv4d8dB88B7fCDfBl2ByeCRWcJKlPsALwjWy7HQWrD/loczgNdui+vR/OKR+WJEXkS5n2OmD5nyHvBZTCVuN2OHhcI/8C28Ob4XjYDH4yvts74Rx417jL+VaCE+G5rH+ZHO6Ds2CbQZ3uCseNW8U4F/